In [1]:
import numpy as np
from tabulate import tabulate

## Datos

In [2]:
# Electronegatividad
X = {"CO": {"Xa": 2.55, "Xb": 3.44}, "NO": {"Xa": 3.04, "Xb": 3.44},
     "SrO": {"Xa": 0.95, "Xb": 3.44}, "MgO": {"Xa": 1.31, "Xb": 3.44}}

# Distancia interatomica en angstroms
d = {"CO": 1.1283, "NO": 1.15077, "SrO": 1.91983, "MgO": 1.749}

# Momento dipolar en D
mu_exp = {"CO": 0.122, "NO": 0.0, "SrO": 8.9, "MgO": 6.2}

In [3]:
# Carga del electron
e = 4.8e-10

## Caracter iónico

### Hann

In [4]:
def pCI_h(Xa, Xb):
    diff = np.abs(Xa-Xb)
    return 16*diff + 3.5*diff**2

In [5]:
pCI_h_ = {molec:pCI_h(**X[molec]) for molec in X}
pCI_h_

{'CO': 17.01235,
 'NO': 6.959999999999998,
 'SrO': 61.540350000000004,
 'MgO': 49.959149999999994}

### Lweis

In [6]:
def pCI_l(Xa, Xb):
    diff = np.abs(Xa-Xb)
    return 100*(1-np.exp(-0.25*diff**2))

In [7]:
pCI_l_ = {molec:pCI_l(**X[molec]) for molec in X}
pCI_l_

{'CO': 17.965065585213402,
 'NO': 3.9210560847676823,
 'SrO': 78.77573323062757,
 'MgO': 67.83286799695526}

## Momento dipolar (utilizando el %CI de Hanney)

In [8]:
# Momento dipolar en D
def mu(pCI, d):
    # delta_AB * d_AB
    delta = (pCI/100.0) * e
    return delta * (d * 1e-8) / (1e-18)

In [9]:
mu_ = {molec:mu(pCI_h_[molec], d[molec]) for molec in d}
mu_ 

{'CO': 0.92136165624,
 'NO': 0.38444924159999994,
 'SrO': 5.671056486744,
 'MgO': 4.194170560800001}

In [10]:
mu_exp

{'CO': 0.122, 'NO': 0.0, 'SrO': 8.9, 'MgO': 6.2}

### Diferencia porcentual $\mu$ vs $\mu_{exp}$

In [11]:
def p_diff(mu, mu_exp):
    if mu_exp == 0:
        return 'NA'
    return "{:0.0f}%".format(100*(mu-mu_exp)/mu_exp)

In [12]:
p_diff_ = {molec:p_diff(mu_[molec], mu_exp[molec]) for molec in mu_}
p_diff_

{'CO': '655%', 'NO': 'NA', 'SrO': '-36%', 'MgO': '-32%'}

----

### Generar tablas LaTeX de resultados

### %CI

In [21]:
table_pCI = []
header_pCI = ["Molécula", "$\%\text{CI}_{H}$", "\%\text{CI}_{L}"]
for molec in X:
    table_pCI.append([molec, "{:0.1f}%".format(pCI_h_[molec]), "{:0.1f}%".format(pCI_l_[molec])])

In [22]:
table_pCI

[['CO', '17.0%', '18.0%'],
 ['NO', '7.0%', '3.9%'],
 ['SrO', '61.5%', '78.8%'],
 ['MgO', '50.0%', '67.8%']]

In [23]:
print(tabulate(table_pCI, headers=header_pCI, tablefmt="latex"))

\begin{tabular}{lll}
\hline
 Molécula   & \$\textbackslash{}\%	ext\{CI\}\_\{H\}\$       & \textbackslash{}\%	ext\{CI\}\_\{L\}       \\
\hline
 CO         & 17.0\% & 18.0\% \\
 NO         & 7.0\%  & 3.9\%  \\
 SrO        & 61.5\% & 78.8\% \\
 MgO        & 50.0\% & 67.8\% \\
\hline
\end{tabular}


### Momento dipolar

In [15]:
table_mu = []
header_mu = ["Molécula", "d", "\mu", "\mu_{exp}", "a"]
for molec in mu_exp:
    table_mu.append([molec, "{}".format(d[molec]),
                     "{:0.2f}".format(mu_[molec]),
                     "{}".format(mu_exp[molec]),
                     p_diff_[molec]
                    ])

In [16]:
table_mu

[['CO', '1.1283', '0.92', '0.122', '655%'],
 ['NO', '1.15077', '0.38', '0.0', 'NA'],
 ['SrO', '1.91983', '5.67', '8.9', '-36%'],
 ['MgO', '1.749', '4.19', '6.2', '-32%']]